# Averaged Artifact Substraction with EEG Data
## First step prepare Jupyter Notebook

In [ ]:
import os, sys
from FACET.Facet import Facet

from loguru import logger

os.getcwd()
os.chdir(os.path.dirname(os.path.abspath("__file__")))

logger.remove()
logger.add(sys.stderr, level="INFO")
logger.add("Facet.log", level="DEBUG")

eegDataSet = "NiazyFMRI.edf"

f = Facet()


## Second Step import EEG Data

In [ ]:
f.import_EEG(eegDataSet, artifact_to_trigger_offset=-0.005, bads=['EMG', 'ECG'])
f.get_EEG().mne_raw.crop(0,162)
f.find_triggers(r'\b1\b')
event_id={'trigger':1}
f.export_EEG(path="./export",fmt="bids", event_id=event_id)
f.import_EEG(path="./export", fmt="bids", artifact_to_trigger_offset=-0.005, bads=['EMG', 'ECG'])

f.plot_EEG(title="after import")

## Third Step Preprocessing

In [ ]:
f.pre_processing()


## Fourth Step find triggers

In [ ]:

f.find_triggers(r'\btrigger\b')
f.align_triggers(0)
#f.cut()

#f.find_triggers(r'.*TR.*') # Using Fischmeisters data

## Fourth step apply AAS

In [ ]:
f.apply_Moosmann(file_path="headmotiondata.tsv", threshold=0.8)

## Fifth step Remove Artifacts

In [ ]:
f.remove_artifacts( )

## Sixth step Post-Processing

In [ ]:
f.downsample()
f.lowpass(70)

## Seventh display the processed eeg data

In [ ]:
f.plot_EEG(title="after lowpass")
f.export_EEG("NiazyFMRI_pyFacet_cleaned.edf")

## Eighth Evaluate Results

In [ ]:
eeg_python = f.get_EEG()
f.add_to_evaluate(eeg_python, name="MNE_matrix") # Matrix will result in >1.0 RMS2 because of python mne inaccuracies

results = f.evaluate(plot=True,measures=["SNR", "RMS", "RMS2", "MEDIAN"])
logger.info(results)

f.export_EEG(path="./export", fmt="bids", event_id=event_id)

